LiNGAMによる因果探索

- 線形な構造方程式モデル（生成モデル）→ Linear
- 誤差ノイズが非ガウス分布（正規分布でない）→ Non Gausian
- 非循環なグラフ→ Acyclic

基本的に，  
x = Bx + e  
という式で表され，Bを解くことが構造方程式を導くことになるので目標となる．  
また，A = (I-B)^-1と置くことで  
x = Aeと表され，これが「独立成分分析」での式と類似しているという特徴がある

ただし，LiNGAMでは行列Bに制約があるため，そうなるようにA^-1(=I-B)を変形する必要がある．  
- 上三角成分が0
- 対角成分が1

In [1]:
# 乱数シードの固定
import random
import numpy as np

random.seed(1234)
np.random.seed(1234)

In [2]:
import pandas as pd

# 自作データ

## model

x1 = 3×x2 + ex1  
x2 = ex2  
x3 = 2×x1 + 4×x2 + ex3

In [10]:
# データ数
num_data = 200

# 非ガウスのノイズ（ポイント!!!!）
ex1 = 2*(np.random.rand(num_data)-0.5) # -1から1
ex2 = 2*(np.random.rand(num_data)-0.5) # -1から1
ex3 = 2*(np.random.rand(num_data)-0.5) # -1から1

# データ生成（構造方程式モデル）
x2 = ex2
x1 = 3*x2 + ex1
x3 = 2*x1 + 4*x2 + ex3

df = pd.DataFrame({"x1": x1, "x2": x2, "x3": x3})

In [11]:
df.head()

,x1,x2,x3
0,1.179586,0.595859,3.915641
1,-2.100976,-0.392811,-5.267171
2,-0.377316,-0.203013,-1.082107
3,-0.174973,-0.101412,-1.124580
4,-3.660327,-0.958930,-10.465744


# 独立成分分析

In [12]:
from sklearn.decomposition import FastICA

ica = FastICA(random_state=1234).fit(df)

# ICAで求めた行列A
A_ica = ica.mixing_

# 行列Aの逆行列を求める
A_ica_inv = np.linalg.pinv(A_ica)

In [13]:
A_ica_inv

array([[-0.25118281, -0.48334517,  0.12477184],
       [ 0.01970723,  0.14798746, -0.00841105],
       [-0.11576958,  0.35454395, -0.00139253]])

## 行列A_inv→制約に合わせて調整

In [14]:
from munkres import Munkres
from copy import deepcopy

In [23]:
# 実装の参考
# [5] Qiita：LiNGAMモデルの推定方法について
# https://qiita.com/m__k/items/bd87c063a7496897ba7c


# ①「行の順番を変換」→対角成分の絶対値を最大にする
# （元のA^-1の対角成分は必ず0ではないので）
# 絶対値の逆数にして対角成分の和を最小にする問題に置き換える
A_ica_inv_small = 1 / np.abs(A_ica_inv)

# 対角成分の和を最小にする行の入れ替え順を求める
m = Munkres() # ハンガリアン法
ixs = np.vstack(m.compute(deepcopy(A_ica_inv_small)))

# 求めた順番で変換
ixs = ixs[np.argsort(ixs[:, 0]), :]
ixs_perm = ixs[:, 1]
A_ica_inv_perm = np.zeros_like(A_ica_inv)
A_ica_inv_perm[ixs_perm] = A_ica_inv
print(A_ica_inv_perm)

[[-0.11576958  0.35454395 -0.00139253]
 [ 0.01970723  0.14798746 -0.00841105]
 [-0.25118281 -0.48334517  0.12477184]]


In [24]:
# 並び変わった順番
print(ixs)

[[0 2]
 [1 1]
 [2 0]]


In [25]:
# ②「行の大きさを調整」
D = np.diag(A_ica_inv_perm)[:, np.newaxis] # D倍されているDを求める
A_ica_inv_perm_D = A_ica_inv_perm / D
print(A_ica_inv_perm_D)

[[ 1.         -3.06249664  0.01202846]
 [ 0.13316823  1.         -0.05683623]
 [-2.01313708 -3.87383229  1.        ]]


In [26]:
# ③「B=I-A_inv」
B_est = np.eye(3) - A_ica_inv_perm_D
print(B_est)

[[ 0.          3.06249664 -0.01202846]
 [-0.13316823  0.          0.05683623]
 [ 2.01313708  3.87383229  0.        ]]


今回の構造方程式は以下なので

x1 = 3x2 + ex1  
x2 = ex2  
x3 = 2x1 + 4x2 + ex3

正解のBは
\begin{pmatrix}
0 & 3 & 0 \\
0 & 0 & 0 \\
2 & 4 & 0 \\
\end{pmatrix}

上記B_estはデータが有限であり計算誤差も生まれてしまうので，0になるべき部分が0でない．  
そこで，0になるべきはずの数の個数分だけ，絶対値が小さい成分を0にする．

In [27]:
# ①上側成分の0になるはずの数（3×3であれば3個、4×4であれば6個と、対角成分の上側の要素数分）、絶対値が小さい成分を0にする
# ②変数の順番を入れ替えて、下三角行列になるかを確かめる、
# 実装の参考
# [5] Qiita：LiNGAMモデルの推定方法について
# https://qiita.com/m__k/items/bd87c063a7496897ba7c

def _slttestperm(b_i):
# b_iの行を並び替えて下三角行列にできるかどうかチェック
    n = b_i.shape[0]
    remnodes = np.arange(n)
    b_rem = deepcopy(b_i)
    p = list() 

    for i in range(n):
        # 成分が全て0である行番号のリスト
        ixs = np.where(np.sum(np.abs(b_rem), axis=1) < 1e-12)[0]

        if len(ixs) == 0:
            return None
        else:
            ix = ixs[0]
            p.append(remnodes[ix])

            #　成分が全て0である行を削除
            remnodes = np.hstack((remnodes[:ix], remnodes[(ix + 1):]))
            ixs = np.hstack((np.arange(ix), np.arange(ix + 1, len(b_rem))))
            b_rem = b_rem[ixs, :]
            b_rem = b_rem[:, ixs]

    return np.array(p)

b = B_est
n = b.shape[0]
assert(b.shape == (n, n))

ixs = np.argsort(np.abs(b).ravel())

for i in range(int(n * (n + 1) / 2) - 1, (n * n) - 1):
    b_i = deepcopy(b)
    b_i.ravel()[ixs[:i]] = 0
    ixs_perm = _slttestperm(b_i)
    if ixs_perm is not None:
        b_opt = deepcopy(b)
        b_opt = b_opt[ixs_perm, :]
        b_opt = b_opt[:, ixs_perm]
        break
b_csl = np.tril(b_opt, -1)
b_csl[ixs_perm, :] = deepcopy(b_csl)
b_csl[:, ixs_perm] = deepcopy(b_csl)

B_est1 = b_csl
print(B_est1)

[[0.         3.06249664 0.        ]
 [0.         0.         0.        ]
 [2.01313708 3.87383229 0.        ]]


これは変数Xの順番を入れ替えると下三角行列になる  
よってこれは非循環の条件を満たしている

# 回帰分析で非ゼロ部分（構造方程式の係数部分）を求め直す

上記で絶対値の小さい部分をゼロ化したので，当操作後での正確な係数を求める

In [32]:
from sklearn.linear_model import LinearRegression

# 説明変数
## 方程式から非ゼロ部分を取り出す
X1 = df[['x2']]
X3 = df[['x1', 'x2']]

# 回帰
reg1 = LinearRegression().fit(X1, df['x1'])
reg3 = LinearRegression().fit(X3, df['x3'])

In [33]:
# 回帰した結果の係数を出力
print("係数：", reg1.coef_)
print("係数：", reg3.coef_)

係数： [2.91955115]
係数： [2.00407234 3.96697864]


この推定値は以下正解とほぼ同じである

x1 = 3x2 + ex1  
x2 = ex2  
x3 = 2x1 + 4x2 + ex3